In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json

ModuleNotFoundError: No module named 'sklearn'

In [2]:
df=pd.read_csv('medical_insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
df=df.rename({'sex':'gender'},axis=1)
df.head()

,age,gender,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df['gender']=df['gender'].replace({'male':0,'female':1})
df['smoker']=df['smoker'].replace({'yes':1,'no':0})
df.head()

,age,gender,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,southwest,16884.92400
1,18,0,33.770,1,0,southeast,1725.55230
2,28,0,33.000,3,0,southeast,4449.46200
3,33,0,22.705,0,0,northwest,21984.47061
4,32,0,28.880,0,0,northwest,3866.85520


In [6]:
df=pd.get_dummies(df,columns=['region'])
df.head()

,age,gender,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,1,27.900,0,1,16884.92400,False,False,False,True
1,18,0,33.770,1,0,1725.55230,False,False,True,False
2,28,0,33.000,3,0,4449.46200,False,False,True,False
3,33,0,22.705,0,0,21984.47061,False,True,False,False
4,32,0,28.880,0,0,3866.85520,False,True,False,False


In [7]:
df['region_northwest']=df['region_northwest'].astype(int)
df['region_northeast']=df['region_northeast'].astype(int)
df['region_southwest']=df['region_southwest'].astype(int)
df['region_southeast']=df['region_southeast'].astype(int)

In [8]:
df.head()

,age,gender,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,1,27.900,0,1,16884.92400,0,0,0,1
1,18,0,33.770,1,0,1725.55230,0,0,1,0
2,28,0,33.000,3,0,4449.46200,0,0,1,0
3,33,0,22.705,0,0,21984.47061,0,1,0,0
4,32,0,28.880,0,0,3866.85520,0,1,0,0


In [9]:
label=[0,1,2,3,4]
df['age_buckets']=pd.cut(df['age'],bins=[0,18,30,45,60,np.inf],labels=label,include_lowest=True,right=False)

In [10]:
df["weight"] = np.where(df["bmi"]>=25,1,0) # 1=overweight,0 normal
df["weight"].value_counts()

weight
1    1093
0     245
Name: count, dtype: int64

In [11]:
df["risky"] = np.where(df["age_buckets"].isin([4]),1,0)# 1 risky, 0 non risky
df["risky"].value_counts()

risky
0    1224
1     114
Name: count, dtype: int64

In [12]:
df.head()

,age,gender,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest,age_buckets,weight,risky
0,19,1,27.900,0,1,16884.92400,0,0,0,1,1,1,0
1,18,0,33.770,1,0,1725.55230,0,0,1,0,1,1,0
2,28,0,33.000,3,0,4449.46200,0,0,1,0,1,1,0
3,33,0,22.705,0,0,21984.47061,0,1,0,0,2,0,0
4,32,0,28.880,0,0,3866.85520,0,1,0,0,2,1,0


In [19]:
df=df.drop('age',axis=1)

In [20]:
x=df.drop('charges',axis=1)
y=df['charges']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=40)

In [21]:
knn_reg=KNeighborsRegressor()
knn_reg.fit(x_train,y_train)

KNeighborsRegressor()

In [22]:
#testing data
knn_y_pred=knn_reg.predict(x_test)
 #MSE
mse_test=mean_squared_error(knn_y_pred,y_test)
print("mse is:\n",mse_test)
mae_test=mean_absolute_error(knn_y_pred,y_test)
print("mae is:\n",mae_test)
r2score_test=knn_reg.score(x_test,y_test)
print("r2_score is:\n",r2score_test)
print()
print(">>>>>>>>>>")

#training data

knn_y_pred=knn_reg.predict(x_train)
 #MSE
mse_train=mean_squared_error(knn_y_pred,y_train)
print("mse is:\n",mse_train)
mae_train=mean_absolute_error(knn_y_pred,y_train)
print("mae is:\n",mae_train)
r2score_train=knn_reg.score(x_train,y_train)
print("r2_score is:\n",r2score_train)

mse is:
 90178191.93739559
mae is:
 6259.001942641791
r2_score is:
 0.3967795258121418

>>>>>>>>>>
mse is:
 59761827.66409669
mae is:
 4980.048536461869
r2_score is:
 0.5900864973098786


In [23]:
#scaling
std_scalar = StandardScaler()
std_scalar.fit(x_train)
array = std_scalar.transform(x_train)
std_train_df = pd.DataFrame(array,columns=x_train.columns)
std_train_df.head(1)

,gender,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest,age_buckets,weight,risky
0,-0.977816,-1.153294,0.748186,-0.508747,-0.563679,-0.569436,-0.603983,1.747292,-1.231152,-2.145249,-0.30488


In [24]:
knn_model = KNeighborsRegressor(n_neighbors=6,p=2) # k=5,p=2
knn_model.fit(std_train_df,y_train)

###### transforming data#######
array = std_scalar.transform(x_test)
std_test_df = pd.DataFrame(array,columns=x_test.columns)

## Testing and training
print(">>>>>>>Testing accuarcy>>>>>")
y_pred = knn_model.predict(std_test_df)
mse = mean_squared_error(y_test,y_pred)
print("MSE\n",mse)

mae = mean_absolute_error(y_test,y_pred)
print("MAE\n",mae)

r2 = r2_score(y_test,y_pred)
print("R2score\n",r2)

#########################
## training
print(">>>>>>>Training accuarcy>>>>>")
y_pred = knn_model.predict(std_train_df)
mse = mean_squared_error(y_train,y_pred)
print("MSE\n",mse)

mae = mean_absolute_error(y_train,y_pred)
print("MAE\n",mae)

r2 = r2_score(y_train,y_pred)
print("R2score\n",r2)

>>>>>>>Testing accuarcy>>>>>
MSE
 33460449.87074736
MAE
 3697.7713439981344
R2score
 0.7761761684955534
>>>>>>>Training accuarcy>>>>>
MSE
 23268284.92574276
MAE
 2994.63316502461
R2score
 0.8404000588952347


In [27]:
#hyperparameter tuning
knn_model=KNeighborsRegressor()
hyper_para={"n_neighbors":np.arange(5,15),"p":[1,2]}
rand_cv=RandomizedSearchCV(knn_model,hyper_para,cv=5)
rand_cv.fit(std_train_df,y_train)
rand_cv.best_estimator_

KNeighborsRegressor(n_neighbors=7)

In [28]:
knn_model=rand_cv.best_estimator_
knn_model.fit(std_train_df,y_train)

## Testing and training
print(">>>>>>>Testing accuarcy>>>>>")
y_pred = knn_model.predict(std_test_df)
mse = mean_squared_error(y_test,y_pred)
print("MSE\n",mse)

mae = mean_absolute_error(y_test,y_pred)
print("MAE\n",mae)

r2 = r2_score(y_test,y_pred)
print("R2score\n",r2)

#########################
## training
print(">>>>>>>Training accuarcy>>>>>")
y_pred = knn_model.predict(std_train_df)
mse = mean_squared_error(y_train,y_pred)
print("MSE\n",mse)

mae = mean_absolute_error(y_train,y_pred)
print("MAE\n",mae)

r2 = r2_score(y_train,y_pred)
print("R2score\n",r2)

>>>>>>>Testing accuarcy>>>>>
MSE
 35147590.79105115
MAE
 3795.5296172270787
R2score
 0.7648905358597392
>>>>>>>Training accuarcy>>>>>
MSE
 25169581.326506898
MAE
 3138.22368463498
R2score
 0.8273588401482121


In [29]:
#Model finalization
with open("KNN_model.pkl","wb") as file:
    pickle.dump(knn_model,file)

with open("scale.pkl","wb") as file:
    pickle.dump(std_scalar,file)

In [30]:
label_encode_data = {"gender":{"female":1,"male":0},
                     "smoker":{"yes":1,"no":0},
                     "columns":list(x.columns)}
label_encode_data

{'gender': {'female': 1, 'male': 0},
 'smoker': {'yes': 1, 'no': 0},
 'columns': ['gender',
  'bmi',
  'children',
  'smoker',
  'region_northeast',
  'region_northwest',
  'region_southeast',
  'region_southwest',
  'age_buckets',
  'weight',
  'risky']}

In [31]:
with open("project_data.json","w") as file:
    json.dump(label_encode_data,file)